# Week 5 exercises

## Part 1: Decision trees (DSFS Chapter 17)

### There are two main kinds of decision trees depending on the type of output (numeric vs. categorical). What are they?

Decision trees are normally divided into classification trees (for categorical outputs) and regression trees (for numeric outputs)

### Explain in your own words: Why is entropy useful when deciding where to split the data?

This is useful because it helps deciding if a decision is filtering out enough candidates. 
The closer the entropy is to 0 the lower the number of likely candidates are. 
The closer to 1 the entropy is the more candidates there are left. 
Therefore we want the ones with the lowest entropy so there is fewer choises in the next step of the tree

### Why are trees prone to overfitting?

Because when training the model to 100% of accuracy, there could be to much irelevante information. So when trying the data on a real set or when testing the accuracy becomes lower then 100%

### Explain (in your own words) how random forests help prevent overfitting.

The forest helps by picking different decisions trees and then voting for the result at the end. This prevents overfitting by making the decission depending on a majority. This helps because the overfitted tree(s) will in most cases be overruled by the majority

### Exercise: Building the minority report algorithm

### Use the category of the crimes to build a decision tree that predicts the corresponding district. You can implement the ID3 tree in the DSFS book, or use the DecisionTreeClassifier class in scikit-learn. For training, you can use 90% of the data and test the tree prediction on the remaining 10%. 

In [10]:
# first we read the data
# import csv utilities and numpy
import csv
import numpy as np

incedents = {}
incedentsSingleDistrict = {}
incedentsDayOfWeek = {}
incedents.update({'data' : [], 'target': []})
incedentsSingleDistrict.update({'data' : [], 'target': []})
incedentsDayOfWeek.update({'data' : [], 'target': []})
with open('SFPD_Incidents_-_from_1_January_2003.csv') as f:
    reader = csv.DictReader(f, delimiter=',')
    for row in reader:
        incedents['data'].append(row['Category'])
        incedents['target'].append(row['PdDistrict'])
        incedentsSingleDistrict['data'].append(row['Category'])
        incedentsSingleDistrict['target'].append(row['PdDistrict'] if (row['PdDistrict']=='MISSION') else 'Other')
        incedentsDayOfWeek['data'].append(row['DayOfWeek'])

In [11]:
# imports
from sklearn import tree
from sklearn.tree import export_graphviz
from __future__ import division

# convert categories and PdDistricts to ints
categories = list(set(incedents['data']))
incedentsX = [categories.index(incedent) for incedent in incedents['data']]
categoriesSingleDistrict = list(set(incedentsSingleDistrict['data']))
incedentsXSingleDistrict = [categories.index(incedent) for incedent in incedentsSingleDistrict['data']]
categoriesDayOfWeek = list(set(incedentsDayOfWeek['data']))
incedentsXDayOfWeek = [categoriesDayOfWeek.index(incedent) for incedent in incedentsDayOfWeek['data']]

# setup the classifiers
clf = tree.DecisionTreeClassifier()
clfSingleDistrict = tree.DecisionTreeClassifier()
clfDayOfWeek = tree.DecisionTreeClassifier()

# split the data into 90% traning and 10% test prediction 
# For all districts
incedentsX_train = np.array(incedentsX[:int(len(incedentsX)*0.9)])
incedentsX_test = np.array(incedentsX[-int(len(incedentsX)*0.1):])
incedentsY_train = np.array(incedents['target'][:int(len(incedentsX)*0.9)])
incedentsY_test = np.array(incedents['target'][-int(len(incedentsX)*0.1):])

# For a single district
incedentsX_trainSingleDistrict = np.array(incedentsXSingleDistrict[:int(len(incedentsXSingleDistrict)*0.9)])
incedentsX_testSingleDistrict = np.array(incedentsXSingleDistrict[-int(len(incedentsXSingleDistrict)*0.1):])
incedentsY_trainSingleDistrict = np.array(incedentsSingleDistrict['target'][:int(len(incedentsXSingleDistrict)*0.9)])
incedentsY_testSingleDistrict = np.array(incedentsSingleDistrict['target'][-int(len(incedentsXSingleDistrict)*0.1):])

# With day of week
incedentsX_trainDayOfWeek = np.array(incedentsXDayOfWeek[:int(len(incedentsXDayOfWeek)*0.9)])
incedentsX_testDayOfWeek = np.array(incedentsXDayOfWeek[-int(len(incedentsXDayOfWeek)*0.1):])

# initialize of new arrays
incedentsX_train_rearranged = np.zeros((len(incedentsX_train), 1))
incedentsX_test_rearranged = np.zeros((len(incedentsX_test), 1))
incedentsX_train_rearrangedSingleDistrict = np.zeros((len(incedentsX_trainSingleDistrict), 1))
incedentsX_test_rearrangedSingleDistrict = np.zeros((len(incedentsX_testSingleDistrict), 1))
incedentsX_train_rearrangedDayOfWeek=[[] for x in range(len(incedentsX_train_rearranged))]
incedentsX_test_rearrangedDayOfWeek=[[] for x in range(len(incedentsX_test_rearranged))]

# Rearrange the training and test data to two dimentional arrays [[features],[features]]
count =0
for x in incedentsX_train:
    incedentsX_train_rearranged[count] = x
    incedentsX_train_rearrangedDayOfWeek[count] = [x,incedentsX_trainDayOfWeek[count]]
    count +=1
    
count =0
for x in incedentsX_test:
    incedentsX_test_rearranged[count] = x
    incedentsX_test_rearrangedDayOfWeek[count] = [x,incedentsX_testDayOfWeek[count]]
    count +=1

count =0
for x in incedentsX_trainSingleDistrict:
    incedentsX_train_rearrangedSingleDistrict[count] = x
    count +=1
    
count =0
for x in incedentsX_testSingleDistrict:
    incedentsX_test_rearrangedSingleDistrict[count] = x
    count +=1

# Train the classifiers and get perdictions from the test data
clf.fit(incedentsX_train_rearranged, incedentsY_train)
Z = clf.predict(incedentsX_test_rearranged)
clfSingleDistrict.fit(incedentsX_train_rearrangedSingleDistrict, incedentsY_trainSingleDistrict)
ZSingleDistrict = clfSingleDistrict.predict(incedentsX_test_rearrangedSingleDistrict)
clfDayOfWeek.fit(incedentsX_train_rearrangedDayOfWeek, incedentsY_train)
ZDayOfWeek = clfDayOfWeek.predict(incedentsX_test_rearrangedDayOfWeek)

What is the fraction of correct predictions?

In [12]:
# Check how many correct perdictions
numbOfCorrect =0
count=0
for ans in Z:
    if ans ==incedentsY_test[count]:
        numbOfCorrect+=1
    count+=1
print "{:0.2f}".format(numbOfCorrect/len(incedentsY_test)*100)

19.20


What are the correct predictions if you restrict the training/prediction to single districts (for example, predicting Mission vs. all other districts, etc)?

In [13]:
# Check how many correct perdictions
numbOfCorrect =0
count=0
for ans in ZSingleDistrict:
    if ans ==incedentsY_testSingleDistrict[count]:
        numbOfCorrect+=1
    count+=1
print "{:0.2f}".format(numbOfCorrect/len(incedentsY_testSingleDistrict)*100)

85.79


Compare it to the random guess, what would you get if you'd guess a district randomly?

In [5]:
from random import randint

def randomGuess(numb):
    if numb==1:
        return "CENTRAL"
    if numb==2:
        return "NORTHERN"
    if numb==3:
        return "PARK"
    if numb==4:
        return "SOUTHERN"
    if numb==5:
        return "MISSION"
    if numb==6:
        return "TENDERLOIN"
    if numb==7:
        return "RICHMOND"
    if numb==8:
        return "TARAVAL"
    if numb==9:
        return "INGLESIDE"
    if numb==10:
        return "BAYVIEW"
    
# Make an array of random guesses
count=0
incedentsX_test_random=["" for x in range(len(incedentsX_test_rearranged))]
for x in incedentsX_test_random:
    incedentsX_test_random[count] = randomGuess(randint(1,10))
    count +=1

# Check how many correct perdictions
numbOfCorrect =0
count=0
for ans in incedentsX_test_random:
    if ans ==incedentsY_test[count]:
        numbOfCorrect+=1
    count+=1
print "{:0.2f}".format(numbOfCorrect/len(incedentsY_test)*100)

10.02


And if you'd guess always one of the districts (for example the district with the most crimes)?

In [6]:
# Make an array of perdictions with the district with the most crimes
incedentsX_test_southern=["SOUTHERN" for x in range(len(incedentsX_test_rearranged))]
    
# Check how many correct perdictions
numbOfCorrect =0
count=0
for ans in incedentsX_test_southern:
    if ans == incedentsY_test[count]:
        numbOfCorrect+=1
    count+=1
print "{:0.2f}".format(numbOfCorrect/len(incedentsY_test)*100)

16.94


Now, add the day of the week to the features, do any of the the performance measures improve?

In [8]:
# Check how many correct perdictions
numbOfCorrect =0
count=0
for ans in ZDayOfWeek:
    if ans ==incedentsY_test[count]:
        numbOfCorrect+=1
    count+=1
print "{:0.2f}".format(numbOfCorrect/len(incedentsY_test)*100)
print "NO THEY DO NOT"

19.11
NO THEY DO NOT


In [6]:
from sklearn.externals.six import StringIO
with open("dot.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f)

## Exercises: I know you read the article above, but just a few questions to make you reflect on the details of the story.

### Explain what features go into the terrorist detection model
The model looks at travel patterns, where people are traveling within a given timeframe but also how often they travel to a location of interest.
They also look at how they use their cell phone, if it is only for incoming calls, they keep on changing their sim cards, how often they turn on/off their phone. They look at their co-travelers, how often they visits airports and overnight trips.

### Which algorithm is used to detect the terrorists?
They are using a random forest algorithm

### Do you agree with the algorithm that Al-Jazeera bureau chief is a good target? Justify your answer.
MAYBE YES NO ???

### What's the size of the training set?
Their training set is only with a relative few "known terrorists"

### Why is it still a problem that the algorithm has a false alam rate at 0.18% at a 50% miss rate?
This means that there would still be thousands of innocent that would be flagged as terrorists and potentially killed.


### Do you have a better grasp of the problems with overfitting after reading this article?
By overfitting you are trying to predict with a low training data on a high test data, this means that there would be a high percentage of false positives.
